In [62]:
import numpy as np
import numpy as np
from numpy import linalg as la
from sklearn import preprocessing as pre
import random
import math
from numpy import ma

In [63]:
#X1(CENTRIST FEATURE):
X11=np.load("F:\lab\code\MLAN\Caltech101(part)\dollar_bill\CENTRIST_feature.npy")
X12=np.load("F:\lab\code\MLAN\Caltech101(part)\Faces\CENTRIST_feature.npy")
X13=np.load("F:\lab\code\MLAN\Caltech101(part)\garfield\CENTRIST_feature.npy")
#X14=np.load("F:\lab\code\MLAN\Caltech101(part)\Motorbikes\CENTRIST_feature.npy")
#X15=np.load("F:\lab\code\MLAN\Caltech101(part)\snoopy\CENTRIST_feature.npy")
#X16=np.load("F:\lab\code\MLAN\Caltech101(part)\stop_sign\CENTRIST_feature.npy")
#X17=np.load("F:\lab\code\MLAN\Caltech101(part)\windsor_chair\CENTRIST_feature.npy")
X1=ma.row_stack((X11,X12,X13))
X1.shape

(521, 254)

In [64]:
#X2(HOG FEATURE):
X21=np.load("F:\lab\code\MLAN\Caltech101(part)\dollar_bill\\HOG_feature.npy")
X22=np.load("F:\lab\code\MLAN\Caltech101(part)\Faces\\HOG_feature.npy")
X23=np.load("F:\lab\code\MLAN\Caltech101(part)\garfield\\HOG_feature.npy")
#X24=np.load("F:\lab\code\MLAN\Caltech101(part)\Motorbikes\HOG_feature.npy")
#X25=np.load("F:\lab\code\MLAN\Caltech101(part)\snoopy\HOG_feature.npy")
#X26=np.load("F:\lab\code\MLAN\Caltech101(part)\stop_sign\HOG_feature.npy")
#X27=np.load("F:\lab\code\MLAN\Caltech101(part)\windsor_chair\HOG_feature.npy")
X2=ma.row_stack((X21,X22,X23))
X2.shape

(521, 1984)

In [83]:
#X3(LBP FEATURE):
X31=np.load("F:\lab\code\MLAN\Caltech101(part)\dollar_bill\\LBP_feature.npy")
X32=np.load("F:\lab\code\MLAN\Caltech101(part)\Faces\\LBP_feature.npy")
X33=np.load("F:\lab\code\MLAN\Caltech101(part)\garfield\\LBP_feature.npy")
#X34=np.load("F:\lab\code\MLAN\Caltech101(part)\Motorbikes\LBP_feature.npy")
#X35=np.load("F:\lab\code\MLAN\Caltech101(part)\snoopy\LBP_feature.npy")
#X36=np.load("F:\lab\code\MLAN\Caltech101(part)\stop_sign\LBP_feature.npy")
#X37=np.load("F:\lab\code\MLAN\Caltech101(part)\windsor_chair\LBP_feature.npy")
X3=ma.row_stack((X31,X32,X33))
X3.shape

(521, 928)

In [77]:
X=[X1,X2,X3]
a=np.array([1,2],dtype=np.integer)
while 1>0:
    break

In [78]:
def EProjSimplex_new(v):
    k=1
    ft=1
    m=v.shape[0]
    v0=v-np.mean(v)+1/m
    vmin=v0.min()
    x=np.zeros((m))
    if vmin<0:
        f=1
        lambda_m=0
        while abs(f)>10**(-10):
            v1=v0-lambda_m
            posidx= (v1>0)
            npos=np.sum(posidx)
            g=-npos
            f=np.sum(v1[posidx])-k
            lambda_m=lambda_m-f/(g+0.1-0.1)
            ft+=1
            if ft>100:
                for i in range(m):
                    if v1[i]>0:
                        x[i]=v1[i]
                break
        for i in range(m):
            if v1[i]>0:
                x[i]=v1[i]
    else:
        x=v0
        
    return x
            
            

In [79]:
def cluster(X,c,num_of_adaptive_neighbours=9):
    
    #Preprocess:
    
    n,_=X[0].shape
    v=len(X)#v:number of different views
    k_=num_of_adaptive_neighbours
    lambda_=random.randint(1,30)
    W=np.ones((v),dtype=np.float32)/v#W records the weight of each view
    for i in range(v):
        X[i]=pre.scale(X[i],axis=1)#You need to import sklearn.preprocessing as pre before using the standardlization module.
    print("Preprocession finished...")
    
    #-------------------------------------------------------
    
    
    #Initialize:
    
    #initialize weighted_distX:
    SUM=np.zeros((n,n),dtype=np.float32)
    distX_initial=np.zeros((n,n),dtype=np.float32)
    for k in range(v):
        for i in range(n):
            for j in range(n):
                distX_initial[i][j]=la.norm(X[k][i]-X[k][j])**2
        SUM=SUM+distX_initial
    distX=1/v*SUM#distX denotes the distance matrix
    
    #initialize S:
    S=np.zeros((n,n),dtype=np.float32)
    rr=np.zeros((n,1),dtype=np.float32)
    ascending_order_index=np.zeros((n,n),dtype=np.integer)
    distXs=np.zeros((n,n),dtype=np.float32)
    for i in range(n):
        ascending_order_index[i]=np.argsort(distX[i])
        distXs[i]=distX[i][ascending_order_index[i]]
    for i in range(n):
        di=distXs[i][1:k_+2]
        rr[i]=0.5*(k_*di[k_]-np.sum(di[:k_]))
        id=ascending_order_index[i][1:k_+2]
        S[i][id]=(di[k_]-di)/(k_*di[k_]-np.sum(di[:k_])+10**(-10))
    alpha=np.mean(rr)
    
    #initialize F:
    S=(S+S.T)/2
    D=np.diag(np.sum(S,axis=1))
    L=D-S
    eigenvalue,eigenvector=la.eig(L)
    sorted_indices = np.argsort(eigenvalue)
    F=eigenvector[:,sorted_indices[:c]]#F denotes the indicator matrix
    
    #-------------------------------------------------------
    
    
    #Update:
    for iter_num in range(30):
        
        #update W:
        for k in range(v):
            tmp=0
            for i in range(n):
                for j in range(n):
                    tmp+=S[i][j]*((la.norm(X[k][i]-X[k][j]))**2)
            W[k]=0.5/(tmp**0.5)
        
        #update S:
        S=np.zeros((n,n),dtype=np.float32)
        ad=np.zeros((n,k_),dtype=np.float32)
        Dist=np.zeros((n,n),dtype=np.float32)
        for i in range(n):
            for j in range(n):
                Dist[i][j]+=lambda_*la.norm(F[i,:]-F[j,:])**2 
                for k in range(v):
                    Dist[i][j]+=W[k]*(la.norm(X[k][i,:]-X[k][j,:]))**2
        for i in range(n):
            idxa0=ascending_order_index[i][1:k_+1]
            S[i][idxa0]=EProjSimplex_new(-Dist[i][idxa0]/(2*alpha))
        
        #update F:
        S=(S+S.T)/2
        D=np.diag(np.sum(S,axis=1))
        L=D-S
        F_old=F
        eigenvalue,eigenvector=la.eig(L)
        sorted_indices = np.argsort(eigenvalue)
        F=eigenvector[:,sorted_indices[:c]]
        
        #update lambda_:
        thre=10**(-10)
        fn1=np.sum(eigenvalue[sorted_indices[:c]])
        fn2=np.sum(eigenvalue[sorted_indices[:c+1]])
        if fn1>thre:
            lambda_=2*lambda_
        elif fn2<thre:
            lambda_=lambda_/2.0
            F=F_old
        else:
            break
            
        #Compute the objective value:
        result=0
        for k in range(v):
            a=0
            for i in range(n):
                for j in range(n):
                    a+=S[i][j]*(la.norm(X[k][i]-X[k][j]))**2
            result+=a**0.5
        result+=alpha*(la.norm(S))**2
        print('After iteration %i,'%(iter_num+1),end='')
        print(' the objective value is %f.'%result)
        
    return S

In [80]:
S=cluster(X,3)

Preprocession finished...
After iteration 1, the objective value is 14041.396079.
After iteration 2, the objective value is 14041.590312.
After iteration 3, the objective value is 14042.450446.
After iteration 4, the objective value is 14044.245956.
After iteration 5, the objective value is 14048.184360.
After iteration 6, the objective value is 14057.583880.
After iteration 7, the objective value is 14083.194346.
After iteration 8, the objective value is 14149.831604.
After iteration 9, the objective value is 14274.235703.
After iteration 10, the objective value is 14449.268751.
After iteration 11, the objective value is 14620.091563.
After iteration 12, the objective value is 14788.941118.
After iteration 13, the objective value is 14819.051167.
After iteration 14, the objective value is 14854.108972.
After iteration 15, the objective value is 14678.606114.


In [74]:
X=np.zeros((8))
X.shape

(8,)

In [ ]:
num=0
for i in range(52):
    for j in range(52):
        if S[i][j]==0:
            num+=1
print(num)